<a id="0"></a> <br>
 # Table of Contents  
1. [Modules and Global Variables](#1)     
1. [Set up](#2)
    1. [Variant Definitions](#3)
1. [Data Preprocessing](#4)
1. [Processing](#5)
1. [Back Projection Processing](#6)
    1. [Step 1](#7)
    1. [Step 2](#8)
    1. [Step 3](#9)
1. [Trimming and Filtering](#10)
    1. [Back-projection Trimming](#11)
    1. [Naive Trimming](#12)
    1. [Filtering](#13)
        1. [Step 1, Naive](#14)
        1. [Step 2, Filter with Masks](#15)
        1. [Step 3, Combine Masks](#16)
        1. [Step 4, Filter Back-Projected](#17)        
1. [Covariance and Inference](#18)

<a id="1"></a> 
## Modules and Global Variables

In [1]:
# import modules and global variables

import os, sys
from imp import reload
import datetime as dt
import shutil
import posixpath as ppath
from copy import deepcopy
import time
import data_processing as dp
from tqdm import tqdm
from collections import defaultdict

print('python version %s' % sys.version)

import numpy as np
print('numpy version %s' % np.__version__)

import pandas as pd
print('pandas version %s' % pd.__version__)

import matplotlib
import matplotlib.pyplot as plt
print('matplotlib version %s' % matplotlib.__version__)

import scipy
from scipy import linalg
print('scipy version %s' % scipy.__version__)

# GLOBAL VARIABLES

# sequence related variables

NUC = ['-', 'A', 'C', 'G', 'T']

# local directories

USER_NAME         = 'YOUR_LOCAL_PATH' 

SARS_DIR          = os.path.join(USER_NAME, 'paper-epi-backprojection/src')
DATA_DIR          = os.path.join(SARS_DIR, 'data-processing-pipeline')
SCRIPT_DIR        = os.path.join(DATA_DIR, 'scripts')
INF_DIR           = os.path.join(DATA_DIR, 'inference-c++')
BP_DIR            = os.path.join(DATA_DIR, 'deconvolution')

# cluster directories

DATA_DATE         = '2024-01-26'
SSH_USER          = 'YOUR_CLUSTER_NAME'
USER_EMAIL        = 'YOUR_EMAIL'
SSH_HOME          = 'YOUR_SSH_SERVER'
SSH_DATA          = ppath.join('YOUR_CLUSTER_DIRECTORY', SSH_USER)
SSH_LMDB          = ppath.join(SSH_DATA, f'lmdb-{DATA_DATE}')

# metadata variables

METADATA_COLS     = [   'accession', 'virus_name',            'date', 'location',             'location_additional', 'submission_date']
METADATA_OCOLS    = ['Accession ID', 'Virus name', 'Collection date', 'Location', 'Additional location information', 'Submission date']
METADATA_XFORM    = [           str,    str.lower,               str,  str.lower,                         str.lower,               str]
METADATA_COMP     = os.path.join(SARS_DIR, f'metadata_tsv_2024_01_26.tar.xz')
METADATA_FILE     = os.path.join(SARS_DIR, f'metadata-{DATA_DATE}', 'metadata.tsv')
METADATA_DIR      = os.path.join(SARS_DIR, f'metadata-{DATA_DATE}')
MSA_NAME          = 'msaCodon_0126.tar.xz'

METADATA_INTERIM  = 'outputmeta.csv'
METADATA_INT_ALT  = 'merged-interim.csv'
MERGED_FINAL      = f'metadata-{DATA_DATE}.csv'
METADATA_NEW      = ppath.join(SSH_DATA, MERGED_FINAL)
REF_TAG           = 'EPI_ISL_402125'
REF_FILE          = f'ref-index-{DATA_DATE}'


%load_ext autoreload
%autoreload 2

python version 3.9.7 (default, Sep 16 2021, 08:50:36) 
[Clang 10.0.0 ]
numpy version 1.20.3
pandas version 1.3.4
matplotlib version 3.4.3
scipy version 1.7.1


<a id="2"></a> 
## Set up

In [7]:
# Transfer files to the cluster

print('scp %s/%s %s &&' % (SCRIPT_DIR, 'data_processing.py', SSH_HOME))

file_paths = os.path.join(SCRIPT_DIR, '*')
print(f'scp -r {file_paths} {SSH_HOME}')

scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/processing-scripts/data_processing.py efinn@cluster.csb.pitt.edu: &&
scp -r /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/processing-scripts/* efinn@cluster.csb.pitt.edu:


In [8]:
# Make files on directory

# updated: list of folders used in processing

print(f'mkdir -p {SSH_DATA} &&')
#print('mkdir -p ./MPL/out &&')
dir_list = [
    'regions-times',
    'msa-reg', 'mutant-counts', 'impute-gaps', 'genome-data', 'genome-unique',
    'genome-trimmed', 'genome-filtered-05', 'genome-max-freqs', 
    'genome-max-mask-05', 'genome-max-combine-mask-05', 
    'genome-filtered-compare-05', 'genome-covar-temp', 'genome-covar', 
    'bp-covar', 'bp-filtered-compare-05', 'bp-format-sh-dir', 'bp-formatted', 'bp-submit-dir', 
    'bp-trim-sh-dir', 'bp-filter-20-05', 'logs', 'scratch', 'scratch-bp', 'bp-trim-20'
]
for name in dir_list:
    dir_path = ppath.join(SSH_DATA, name)
    if name!=dir_list[-1]:
        print(f'mkdir {dir_path} &&')
    else:
        print(f'mkdir {dir_path}')

mkdir -p /net/dali/home/barton/efinn/deconvolution-new &&
mkdir -p ./MPL/out &&
mkdir /net/dali/home/barton/efinn/deconvolution-new/regions-times &&
mkdir /net/dali/home/barton/efinn/deconvolution-new/msa-reg &&
mkdir /net/dali/home/barton/efinn/deconvolution-new/genome-data &&
mkdir /net/dali/home/barton/efinn/deconvolution-new/genome-trimmed &&
mkdir /net/dali/home/barton/efinn/deconvolution-new/genome-filtered &&
mkdir /net/dali/home/barton/efinn/deconvolution-new/max-freqs &&
mkdir /net/dali/home/barton/efinn/deconvolution-new/freqs &&
mkdir /net/dali/home/barton/efinn/deconvolution-new/counts &&
mkdir /net/dali/home/barton/efinn/deconvolution-new/trajectories &&
mkdir /net/dali/home/barton/efinn/deconvolution-new/infer


<a id="3"></a> 
### Variant definitions

In [10]:
# Variant definitions

epsilon        = np.unique(['NSP13-260-0-T', 'S-13-1-T', 'S-152-2-T', 'NC-28271-T', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G', 
                  'S-452-1-G', 'N-205-1-T', 'ORF3a-57-2-T', 'NSP9-65-0-G']) # Done

alpha          = np.unique(['NSP6-106-0--', 'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--',
                  'NSP6-108-1--', 'NSP6-108-2--', 'S-501-0-T', 'NSP12-412-2-T', 'NSP2-36-2-T', 'NSP3-183-1-T', 'NSP3-890-1-A', 
                  'NSP3-1089-2-T', 'NSP3-1412-1-C', 'NSP12-613-2-T', 'NSP12-912-2-C', 'S-68-1--', 'S-68-2--', 'S-69-0--', 'S-69-1--', 
                  'S-69-2--', 'S-70-0--', 'S-143-2--', 'S-144-0--', 'S-144-1--', 'S-570-1-A', 'S-681-1-A', 'S-716-1-T', 'S-982-0-G', 
                  'S-1118-0-C', 'ORF8-27-0-T', 'ORF8-52-1-T', 'ORF8-73-1-G', 'N-3-0-C', 'N-3-1-T', 'N-3-2-A', 'N-235-1-T',
                  'N-203-1-A', 'N-203-2-A', 'N-204-0-C', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G'])    # Done

# NEED TO ADD S:D138Y
gamma          = np.unique(['NSP9-31-2-T', 'NSP1-156-2-C', 'NSP3-10-2-T', 'NSP3-370-1-T', 'NSP3-977-0-C', 'NSP3-1200-2-G', 'NSP3-1298-2-G', 
                  'NSP12-140-2-T', 'NSP13-341-2-T', 'S-20-1-A', 'S-417-1-C', 'S-1027-1-T', 'ORF3a-253-0-C', 'ORF8-92-0-A', 'N-80-1-G', 
                  'S-190-2-T', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G', 'S-501-0-T', 'NSP6-106-0--', 'NSP6-106-1--', 
                  'NSP6-106-2--', 'NSP6-107-0--', 'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--', 'NSP6-108-1--', 'NSP6-108-2--',
                  'N-203-1-A', 'N-203-2-A', 'N-204-0-C', 'N-202-0-T', 'N-202-1-C', 'S-18-0-T', 'S-26-0-T', 'S-655-0-T',
                  'S-1176-0-T'])    # Done

twentyE_EU1    = np.unique(['NSP16-199-2-C', 'NSP1-60-2-C', 'NSP3-1189-2-T', 'M-93-2-G', 'N-220-1-T', 'ORF10-30-0-T', 'S-222-1-T',
                  'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G'])    # Done
# This is Pango lineage B.1.177

delta          = np.unique(['NSP12-671-0-A', 'NC-209-T', 'NSP13-77-1-T', 'S-19-1-G', 'S-156-1--', 'S-156-2--', 'S-157-0--', 'S-157-1--', 'S-157-2--', 
                  'S-158-0--', 'S-478-1-A', 'S-681-1-G', 'S-950-0-A', 'ORF3a-26-1-T', 'M-82-1-C', 'ORF7a-82-1-C', 'ORF7a-120-1-T', 
                  'ORF8-119-0--', 'ORF8-119-1--', 'ORF8-119-2--', 'ORF8-120-0--', 'ORF8-120-1--', 'ORF8-120-2--', 'N-63-1-G', 'N-203-1-T', 
                  'N-377-0-T', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G', 'S-452-1-G', 'NC-28270--', 'NSP4-492-1-T'])    # Done

beta           = np.unique(['S-80-1-C', 'NSP3-837-2-T', 'S-215-1-G', 'E-71-1-T', 'S-240-2--', 'S-241-0--', 'S-242-1--', 'S-242-2--', 'S-243-0--', 
                  'S-240-1--', 'S-241-1--', 'S-241-2--', 'S-242-0--', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G', 'S-501-0-T',
                  'NSP6-106-0--', 'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--', 
                  'NSP6-108-1--', 'NSP6-108-2--', 'S-417-2-T', 'S-484-0-A', 'S-701-1-T', 'ORF3a-57-2-T', 'NSP2-85-1-T', 'NSP5-90-1-G',
                  'N-205-1-T', 'NC-173-T', 'ORF8-120-2-T'])    # Done

lambda_new     = np.unique(['S-246-2--', 'NSP3-1569-0-G', 'S-247-0--', 'S-247-1--', 'S-247-2--', 'S-248-0--', 'S-248-1--', 'S-248-2--', 'S-249-0--', 
                  'S-249-1--', 'S-249-2--', 'S-250-0--', 'S-250-1--', 'S-250-2--', 'S-251-0--', 'S-251-1--', 'S-251-2--', 'S-252-0--', 
                  'S-246-1--', 'S-252-1--', 'S-252-2--', 'S-253-0--', 'N-214-0-T', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G', 
                  'NSP6-106-0--', 'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--', 
                  'NSP6-108-1--', 'NSP6-108-2--', 'S-75-1-T', 'S-76-1-T', 'S-452-1-A', 'S-490-1-C', 'S-859-1-A', 'NSP3-428-1-T',
                  'NSP3-1469-0-T', 'NSP4-438-1-C', 'NSP4-492-1-T', 'NSP5-15-0-A']) # Done

iota           = np.unique(['N-234-2-A', 'S-5-0-T', 'S-95-1-T', 'S-253-1-G', 'S-484-0-A', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G',
                 'S-701-1-T', 'NSP13-88-2-C', 'ORF3a-42-1-T', 'ORF3a-67-2-T', 'NSP2-85-1-T', 'NSP4-438-1-C', 'NSP6-106-0--', 'NSP6-106-1--', 
                 'NSP6-106-2--', 'NSP6-107-0--', 'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--',  'NSP6-108-1--', 'NSP6-108-2--', 'N-199-1-T',
                 'N-232-2-A', 'ORF8-11-1-T', 'NSP15-214-2-G', 'NC-28270--'])

D614G          = ['NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G']    # Done

B1_1_318       = np.unique(['NSP15-320-0-A', 'S-575-2-C', 'S-1238-2-A', 'ORF7b-44-2--', 'NC-27887--', 'NC-27888--', 'NC-27889--', 'NC-27890--', 
                  'NC-27891--', 'NC-27892--', 'ORF8-1-0--', 'ORF8-1-1--', 'ORF8-1-2--', 'NSP4-173-1-T', 'S-796-0-C', 'ORF8-2-0--', 
                  'ORF8-2-1--', 'ORF8-2-2--', 'ORF8-3-0--', 'ORF8-3-1--', 'NC-28270-G', 'N-208-2--', 'N-209-0--', 'N-208-1--',
                  'NSP3-378-1-T', 'NSP3-1693-2-T', 'NSP5-21-1-T', 'NSP6-106-0--', 'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 
                  'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--', 'NSP6-108-1--', 'NSP6-108-2--', 'S-95-1-T', 'S-143-2--', 'S-144-0--', 
                  'S-144-1--', 'S-484-0-A', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G', 'S-681-1-A', 'M-82-1-C',
                  'N-203-1-A', 'N-203-2-A', 'N-204-0-C']) # Add N-234-2-T, Many others

# need to add orf1b-314, 3037
omicron        = np.unique(['NC-240-T', 'NSP3-38-1-G', 'NSP3-106-2-T', 'NSP3-889-2-G', 'NSP3-1265-1--', 'NSP3-1265-2--', 'NSP3-1266-0--', 
                  'NSP3-1892-0-A', 'NSP4-492-1-T', 'NSP5-132-1-A', 'NSP6-104-1--', 'NSP6-104-2--', 'NSP6-105-0--', 'NSP6-105-1--', 
                  'NSP6-105-2--', 'NSP6-106-0--', 'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 'NSP6-189-0-G', 'NSP10-57-2-C', 
                  'NSP12-323-1-T', 'NSP12-600-2-T', 'S-67-1-T', 'S-68-1--', 'S-68-2--', 'S-69-0--', 'S-69-1--', 
                  'S-69-2--', 'S-70-0--', 'S-95-1-T', 'S-142-1--', 'S-142-2--', 'S-143-0--', 'S-143-1--', 'S-143-2--', 'S-144-0--', 
                  'S-144-1--', 'S-144-2--', 'S-145-0--', 'S-211-1--', 'S-211-2--', 'S-212-0--', 'S-339-1-A', 'S-371-0-C', 
                  'S-371-1-T', 'S-373-0-C', 'S-417-2-T', 'S-440-2-G', 'S-446-0-A', 'S-547-1-A', 'S-614-1-G', 'S-655-0-T', 
                  'S-679-2-G', 'S-681-1-A', 'S-764-2-A', 'S-796-0-T', 'S-856-2-A', 'S-954-2-T', 'S-969-2-A', 'S-981-0-T', 
                  'S-1146-2-T', 'ORF3a-64-2-T', 'E-9-1-T', 'M-3-1-G', 'M-19-0-G', 'M-63-0-A', 'ORF6-20-0-C', 
                  'NC-28270-T', 'N-13-1-T', 'N-30-1--', 'N-30-2--', 'N-31-0--', 'N-31-1--', 'N-31-2--', 'N-32-0--', 'N-32-1--', 
                  'N-32-2--', 'N-33-0--', 'N-203-1-A', 'N-203-2-A', 'N-204-0-C', 'S-375-1-T', 'S-477-1-A', 'S-478-1-A', 
                  'S-484-1-C', 'S-493-1-G', 'S-496-0-A', 'S-498-1-G', 'S-501-0-T', 'S-505-0-C', 'NSP14-42-0-G',
                  'ORF7b-18-0-T', 'S-214m-2-G', 'S-214n-2-A', 'S-214o-2-G', 'S-214p-2-C', 'S-214q-2-A', 'S-214r-2-A', 
                  'S-214s-2-G', 'S-214t-2-A', 'S-214u-2-A'])  

# 3037
ba2            = np.unique(['S-339-1-A', 'S-371-1-T', 'S-373-0-C', 'S-417-2-T', 'S-440-2-G', 'S-655-0-T', 
                  'S-679-2-G', 'S-681-1-A', 'S-764-2-A', 'S-796-0-T', 'S-954-2-T', 'S-969-2-A',
                  'N-13-1-T', 'N-30-1--', 'N-30-2--', 'N-31-0--', 'N-31-1--', 'N-31-2--', 'N-32-0--', 'N-32-1--', 
                  'N-32-2--', 'N-33-0--', 'N-203-1-A', 'N-203-2-A', 'N-204-0-C', 'NSP4-492-1-T', 'NSP5-132-1-A', 
                  'NSP6-104-1--', 'NSP6-104-2--', 'NSP6-105-0--', 'NSP6-105-1--', 'NSP6-105-2--', 'NSP6-106-0--', 
                  'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 'E-9-1-T', 'ORF3a-223-1-T',
                  'M-19-0-G', 'M-63-0-A', 'S-19-1-T', 'S-24-1--', 'S-24-2--', 'S-25-0--', 'S-25-1--', 'S-25-2--',
                  'S-26-0--', 'S-26-1--', 'S-26-2--', 'S-27-0--', 'S-142-1-A', 'S-213-1-G', 
                  'S-375-1-T', 'S-376-0-G', 'S-408-2-C', 'S-477-1-A', 'S-478-1-A', 'S-484-1-C', 'S-493-1-G', 
                  'S-498-1-G', 'S-501-0-T', 'S-505-0-C', 'N-413-0-C', 'NSP13-392-0-T', 'NSP14-42-0-G',
                  'NSP15-112-1-T', 'ORF6-61-0-C', 'ORF6-61-1-T', 'ORF6-61-2-C', 'S-1146-2-T',
                  'NSP3-534-2-T', 'NSP4-290-2-G', 'NSP5-48-2-T', 'NSP5-131-2-A', 'NSP9-65-2-T', 'NSP12-758-2-T',
                  'NSP15-145-2-G', 'ORF3a-64-2-T', 'M-112-2-T', 'ORF6-20-0-C', 'ORF7b-18-0-T', 'NC-28270-T', 
                  'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G', 'S-405-0-A', 'S-214m-2-G', 
                  'S-214n-2-A', 'S-214o-2-G', 'S-214p-2-C', 'S-214q-2-A', 'S-214r-2-A', 'S-214s-2-G', 
                  'S-214t-2-A', 'S-214u-2-A', 'NSP1-135-2-G', 'NSP3-24-1-T', 'NSP3-489-0-A', 'NSP4-264-0-T', 
                  'NSP4-327-1-T'])

# deletion after 29734
ba4            = np.unique(['S-339-1-A', 'S-371-1-T', 'S-373-0-C', 'S-417-2-T', 'S-440-2-G', 'S-655-0-T', 
                  'S-679-2-G', 'S-681-1-A', 'S-764-2-A', 'S-796-0-T', 'S-954-2-T', 'S-969-2-A',
                  'S-19-1-T', 'S-24-1--', 'S-24-2--', 'S-25-0--', 'S-25-1--', 'S-25-2--', 'S-452-1-G',
                  'S-26-0--', 'S-26-1--', 'S-26-2--', 'S-27-0--', 'S-142-1-A', 'S-213-1-G', 
                  'S-375-1-T', 'S-376-0-G', 'S-408-2-C', 'S-477-1-A', 'S-478-1-A', 'S-484-1-C',
                  'S-498-1-G', 'S-501-0-T', 'S-505-0-C', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 
                  'S-614-1-G', 'S-405-0-A', 'S-214m-2-G', 'S-214n-2-A', 'S-214o-2-G', 
                  'S-214p-2-C', 'S-214q-2-A', 'S-214r-2-A', 'S-214s-2-G', 'S-214t-2-A', 'S-214u-2-A',
                  'S-68-1--', 'S-68-2--', 'S-69-0--', 'S-69-1--', 'S-69-2--', 'S-70-0--', 'NSP1-135-2-G',
                  'N-13-1-T', 'N-30-1--', 'N-30-2--', 'N-31-0--', 'N-31-1--', 'N-31-2--', 'N-32-0--', 
                  'N-32-1--', 'N-32-2--', 'N-33-0--', 'N-203-1-A', 'N-203-2-A', 'N-204-0-C',
                  'NSP3-24-1-T', 'NSP3-489-0-A', 'NSP4-264-0-T', 'NSP4-492-1-T', 'NSP4-327-1-T',
                  'NSP6-104-1--', 'NSP6-104-2--', 'NSP6-105-0--', 'NSP6-105-1--', 'NSP6-105-2--', 'NSP6-106-0--', 
                  'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 'NSP5-132-1-A', 'NSP13-392-0-T', 'NSP14-42-0-G',
                  'NSP15-145-2-G', 'ORF3a-223-1-T', 'M-19-0-G', 'M-63-0-A', 'E-9-1-T',
                  'NSP3-534-2-T', 'NSP4-290-2-G', 'NSP5-48-2-T', 'NSP5-131-2-A', 'NSP9-65-2-T', 'NSP12-758-2-T',
                  'NSP15-145-2-G', 'ORF3a-64-2-T', 'M-112-2-T', 'ORF6-20-0-C', 'ORF7b-18-0-T', 'NC-28270-T',
                  'S-1146-2-T', 'ORF7b-11-2-T', 'S-486-0-G', 'NSP8-23-2-A', 'N-151-0-T',
                  'NSP1-141-0--', 'NSP1-141-1--', 'NSP1-141-2--', 'NSP1-142-0--', 'NSP1-142-1--',
                  'NSP1-142-2--', 'NSP1-143-0--', 'NSP1-143-1--', 'NSP1-143-2--', 'N-413-0-C',
                  'ORF6-61-0-C', 'ORF6-61-1-T', 'ORF6-61-2-C'])

# deletion after 29734
ba5            = np.unique(['S-339-1-A', 'S-371-1-T', 'S-373-0-C', 'S-417-2-T', 'S-440-2-G', 'S-655-0-T', 
                  'S-679-2-G', 'S-681-1-A', 'S-764-2-A', 'S-796-0-T', 'S-954-2-T', 'S-969-2-A',
                  'S-19-1-T', 'S-24-1--', 'S-24-2--', 'S-25-0--', 'S-25-1--', 'S-25-2--', 'S-452-1-G',
                  'S-26-0--', 'S-26-1--', 'S-26-2--', 'S-27-0--', 'S-142-1-A', 'S-213-1-G', 
                  'S-375-1-T', 'S-376-0-G', 'S-408-2-C', 'S-477-1-A', 'S-478-1-A', 'S-484-1-C',
                  'S-498-1-G', 'S-501-0-T', 'S-505-0-C', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 
                  'S-614-1-G', 'S-405-0-A','S-214m-2-G', 'S-214n-2-A', 'S-214o-2-G', 'S-214p-2-C', 
                  'S-214q-2-A', 'S-214r-2-A', 'S-214s-2-G', 'S-214t-2-A', 'S-214u-2-A',
                  'S-68-1--', 'S-68-2--', 'S-69-0--', 'S-69-1--', 'S-69-2--', 'S-70-0--', 'NSP1-135-2-G',
                  'N-13-1-T', 'N-30-1--', 'N-30-2--', 'N-31-0--', 'N-31-1--', 'N-31-2--', 'N-32-0--', 
                  'N-32-1--', 'N-32-2--', 'N-33-0--', 'N-203-1-A', 'N-203-2-A', 'N-204-0-C',
                  'NSP3-24-1-T', 'NSP3-489-0-A', 'NSP4-264-0-T', 'NSP4-492-1-T', 'NSP4-327-1-T',
                  'NSP6-104-1--', 'NSP6-104-2--', 'NSP6-105-0--', 'NSP6-105-1--', 'NSP6-105-2--', 'NSP6-106-0--', 
                  'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 'NSP5-132-1-A', 'NSP13-392-0-T', 'NSP14-42-0-G',
                  'NSP3-534-2-T', 'NSP4-290-2-G', 'NSP5-48-2-T', 'NSP5-131-2-A', 'NSP9-65-2-T', 'NSP12-758-2-T',
                  'NSP15-145-2-G', 'ORF3a-223-1-T', 'M-19-0-G', 'M-63-0-A', 'E-9-1-T', 'M-3-0-A', 'S-486-0-G',
                  'NSP8-23-2-A', 'N-413-0-C', 'S-1146-2-T', 'NC-28270-T', 'ORF3a-64-2-T', 'ORF7b-18-0-T'])

ba2121         = np.unique(['S-339-1-A', 'S-371-1-T', 'S-373-0-C', 'S-417-2-T', 'S-440-2-G', 'S-655-0-T', 
                  'S-679-2-G', 'S-681-1-A', 'S-764-2-A', 'S-796-0-T', 'S-954-2-T', 'S-969-2-A',
                  'S-19-1-T', 'S-24-1--', 'S-24-2--', 'S-25-0--', 'S-25-1--', 'S-25-2--', 'S-452-1-G',
                  'S-26-0--', 'S-26-1--', 'S-26-2--', 'S-27-0--', 'S-142-1-A', 'S-213-1-G', 
                  'S-375-1-T', 'S-376-0-G', 'S-408-2-C', 'S-477-1-A', 'S-478-1-A', 'S-484-1-C',
                  'S-498-1-G', 'S-501-0-T', 'S-505-0-C', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 
                  'S-614-1-G', 'S-405-0-A', 'S-214m-2-G', 'S-214n-2-A', 'S-214o-2-G', 'S-214p-2-C', 
                  'S-214q-2-A', 'S-214r-2-A', 'S-214s-2-G', 'S-214t-2-A', 'S-214u-2-A',
                  'S-68-1--', 'S-68-2--', 'S-69-0--', 'S-69-1--', 'S-69-2--', 'S-70-0--', 'S-452-1-A',
                  'N-13-1-T', 'N-30-1--', 'N-30-2--', 'N-31-0--', 'N-31-1--', 'N-31-2--', 'N-32-0--', 'N-32-1--', 
                  'N-32-2--', 'N-33-0--', 'N-203-1-A', 'N-203-2-A', 'N-204-0-C', 'NSP4-492-1-T', 'NSP5-132-1-A', 
                  'NSP6-104-1--', 'NSP6-104-2--', 'NSP6-105-0--', 'NSP6-105-1--', 'NSP6-105-2--', 'NSP6-106-0--', 
                  'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 'NSP6-189-0-G', 'E-9-1-T', 'ORF3a-223-1-T',
                  'M-19-0-G', 'M-63-0-A', 'N-413-0-C', 'NSP13-392-0-T', 'NSP14-42-0-G',
                  'NSP15-112-1-T', 'ORF6-61-0-C', 'ORF6-61-1-T', 'ORF6-61-2-C', 'S-1146-2-T',
                  'NSP3-534-2-T', 'NSP4-290-2-G', 'NSP5-48-2-T', 'NSP5-131-2-A', 'NSP9-65-2-T', 'NSP12-758-2-T',
                  'NSP15-145-2-G', 'ORF3a-64-2-T', 'M-112-2-T', 'ORF6-20-0-C', 'ORF7b-18-0-T', 'NC-28270-T',
                  'NSP1-135-2-G', 'NSP3-24-1-T', 'NSP3-489-0-A', 'NSP4-264-0-T', 'NSP4-327-1-T', 
                  'S-704-1-T', 'S-452-1-A'])
 
# synonymous: 9593T
eta            = np.unique(['NSP2-231-2-T', 'NSP2-334-2-G', 'NSP2-618-2-A', 'NSP4-13-2-C', 'NSP16-22-2-G',
                  'S-888-0-C', 'ORF6-2-0--', 'ORF6-2-1--', 'ORF6-2-2--', 'N-2-1--', 'N-2-2--',
                  'N-3-0--', 'N-12-1-G', 'N-142-2-G', 'S-68-1--', 'S-68-2--', 'S-69-0--', 'S-69-1--', 
                  'S-69-2--', 'S-70-0--', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G',
                  'M-82-1-C', 'NSP6-106-0--', 'NSP6-106-1--', 'NSP6-106-2--', 'NSP6-107-0--', 
                  'NSP6-107-1--', 'NSP6-107-2--', 'NSP6-108-0--',  'NSP6-108-1--', 'NSP6-108-2--',
                  'S-52-1-G', 'S-67-1-T', 'S-143-2--', 'S-144-0--', 'S-144-1--', 'S-484-0-A', 
                  'S-677-2-C', 'N-205-1-T', 'E-21-0-T', 'NSP3-1189-1-T',
                  'NSP14-44-2-T', 'S-1062-2-C', 'N-142-2-G', 'NC-29542-T'])

# confused about orf1a-T1567I
kappa           = np.unique(['NSP12-323-1-T', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G',
                   'S-154-0-A', 'S-1071-2-T', 'S-681-1-G', 'S-484-0-C', 'S-452-1-G', 'NSP13-206-0-T',
                   'NSP13-429-2-T', 'NSP15-259-1-G', 'NSP15-261-0-G', 'N-203-1-T', 'N-377-0-T',
                   'M-82-1-G', 'ORF3a-26-1-T', 'NSP3-749-1-T', 'NSP6-77-0-G', 'ORF7a-82-1-C',
                   'NC-209-T', 'NSP3-246-2-T', 'M-53-2-T', 'NC-28270--'])

# maybe S-144-0-A, S-144-1-C, S-144-2-C, S-145-0-A (actually there is an insertion here)
# not positive about 'NSP3-237-0-G'
mu              = np.unique(['S-346-2-C', 'NC-240-T', 'NSP3-106-2-T', 'NSP12-323-1-T', 'S-614-1-G', 'S-95-1-T',
                   'S-484-1-C', 'S-501-0-T', 'S-681-1-A', 'S-950-0-A', 'NSP3-237-0-G', 'NSP3-720-1-T',
                   'NSP4-492-1-T', 'NSP6-160-1-G', 'NSP12-323-1-T', 'NSP13-419-0-T', 'N-205-1-T',
                   'ORF3a-57-2-T', 'ORF3a-256-0--', 'ORF3a-256-1--', 'ORF3a-256-2--', 'ORF3a-257-0--',
                   'ORF3a-257-2-C', 'ORF8-11-1-T', 'ORF8-38-0-T', 'ORF8-67-1-T',
                   'NSP3-1106-2-T', 'NSP10-11-2-T', 'NSP14-280-0-T', 'NSP15-176-2-T', 'NC-26491-T',
                   'NC-28271-T'])

all_variants   = [epsilon, alpha, beta, gamma, lambda_new, delta, twentyE_EU1, D614G, omicron, ba2, ba5, ba4, ba2121]
np.save(os.path.join(DATA_DIR, 'variants.npy'), all_variants)

var_dic = {
    'alpha' : alpha, 
    'epsilon' : epsilon, 
    'beta' : beta, 
    'gamma' : gamma, 
    'lambda' : lambda_new, 
    'delta' : delta, 
    '20E\(EU1\)' : twentyE_EU1, 
    'B.1' : D614G,
    'omicron' : omicron,
    'BA.2' : ba2,
    'eta' : eta,
    'iota' : iota,
    'B.1.1.318' : B1_1_318,
    'kappa' : kappa,
    'mu' : mu,
    'BA.4' : ba4,
    'BA.5' : ba5,
    'BA.2.12.1' : ba2121
}

variants_full = [var_dic[i] for i in var_dic]
np.save(os.path.join(DATA_DIR, 'variants-all.npy'), variants_full)

np.save(os.path.join(DATA_DIR, 'alpha-mutations.npy'),   alpha)
np.save(os.path.join(DATA_DIR, 'gamma-mutations.npy'),   gamma)
np.save(os.path.join(DATA_DIR, 'delta-mutations.npy'),   delta)
np.save(os.path.join(DATA_DIR, 'omicron-mutations.npy'), omicron)

# Saving the order in which the variants appeared for later early detection analysis
group_order   = ['B.1', 'Alpha', 'Beta', 'Epsilon', 'Gamma', 'Eta', 'Iota', 'Kappa', 'Lambda', 'Mu', 'Delta', 'BA.1', 'BA.2', 'BA.2.12.1', 'BA.4', 'BA.5']
variant_order = [[D614G], [alpha, beta, epsilon], [gamma, eta, iota, kappa, lambda_new], [mu], [delta], [omicron], [ba2], [ba2121, ba4, ba5]]

new_groups = []
old_muts   = []
for i in range(len(variant_order)):
    current_muts = []
    for group in variant_order[i]:
        new = [j for j in group if j not in old_muts]
        new_groups.append(new)
        for j in new:
            current_muts.append(j)
    for j in current_muts:
        if j not in old_muts:
            old_muts.append(j)
            
np.save(os.path.join(SARS_DIR, 'variants-exclusive.npy'), new_groups)

# THE BELOW ARE INCOMPLETE LISTS OF MUTATIONS
b12    = ['NSP5-89-0-T', 'NSP14-129-0-G', 'ORF3a-172-1-T', 'ORF8-24-1-T', 'N-67-0-T']    # this is B.1.2
b11318 = ['NSP15-320-0-A', 'S-575-2-C', 'S-1238-2-A', 'ORF7b-44-2--', 'NC-27887--', 'NC-27888--', 'NC-27889--', 'NC-27890--',     # This is B.1.1.318, S:D796H
          'NC-27891--', 'NC-27892--', 'ORF8-1-0--', 'ORF8-1-1--', 'ORF8-1-2--', 'NSP4-173-1-T', 'S-796-0-C', 'ORF8-2-0--', 
          'ORF8-2-1--', 'ORF8-2-2--', 'ORF8-3-0--', 'ORF8-3-1--', 'NC-28270-G', 'N-208-2--', 'N-209-0--', 'N-208-1--']
b1429  = ['NSP8-3-2-T', 'NSP2-598-0-C', 'NSP4-131-2-T', 'NSP9-65-0-G', 'S-929-2-C', 'NC-27889-T', 'NSP2-530-2-T']


# ADD ZETA, THETA

<a id="4"></a> 
## Data preprocessing

#### extracting accessions, virus name, date, and location information from metadata

In [11]:
print(f'mkdir {METADATA_DIR} &&')
print(f'tar -xf {METADATA_COMP} -C {METADATA_DIR}')

mkdir /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/metadata-2024-01-26 &&
tar -xf /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/metadata_tsv_2023_06_05.tar.xz -C /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/metadata-2024-01-26


In [ ]:
#Extracting accessions

script_file      = 'subset-metadata.py'

job_file         = 'metaprocess.sh'
job_str          = f"""#!/bin/bash -l

#SBATCH --job-name=metaprocess
#SBATCH --output=metaprocess.log
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --partition=big_memory
#SBATCH --mem=32G
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH --time=6:00:00

echo Running on `hostname`
echo workdir $SLURM_SUBMIT_DIR

#input_directory = "{SSH_DATA}"
#cd input_directory

cd $SLURM_SUBMIT_DIR

#scratch drive folder
SCRDIR=/scr/${SLURM_JOB_ID}

if [[ ! -e $SCRDIR ]]; then
	mkdir $SCRDIR

fi

chmod +rX $SCRDIR

echo scratch drive ${SCRDIR}

cd $SCRDIR

cp $SLURM_SUBMIT_DIR/subset-metadata.py ${SCRDIR}
cp $SLURM_SUBMIT_DIR/metadata.tsv ${SCRDIR}


# load and activate conda environment
module load anaconda
#conda activate align-env
conda activate sars-env
import pandas as pd 

# input and output files

input_tsv="metadata.tsv"
output_tsv="outputmeta.tsv"
output_csv="outputmeta.csv"

#copy files back to working dir on exit
trap "mv outputmeta.tsv $SLURM_SUBMIT_DIR" EXIT
trap "mv outputmeta.csv $SLURM_SUBMIT_DIR" EXIT

# Redirect stdout and stderr to the log file
exec > metaprocess.log 2>&1

# Print job information
echo "SLURM job ID: $SLURM_JOB_ID"
echo "Start time: $(date)"

# Run the Python script
python subset_metadata.py "$input_tsv" "$output_tsv"

echo "metaprocess completed!"
echo "Saved to: $output_tsv"

csv_table=pd.read_table(output_tsv,sep='\t')
csv_table.to_csv('output_csv',index=False)

# Print job completion information
echo "conversion completed!"
echo "Saved to: $output_csv"
"""

f = open(os.path.join(SARS_DIR, job_file), mode='w')
f.write('%s\n' % job_str)
f.close()

print('scp %s/%s %s '   % (SARS_DIR, job_file, SSH_HOME))
print('')

# commands to execute on the cluster

print('sbatch %s' % job_file)
print('')

#### Un-tar file

In [ ]:
msa_file2 = f'{msa_name}.tar.xz'
msa_path2 = os.path.join(SARS_DIR, msa_file2)
tar -xf {SSH_DATA}/{msa_file2} -C {SSH_DATA}

#### merging sequence data with metadata

In [4]:
# Merging sequence data in GISAID alignment with metadata on the cluster

# Write sequences to lightning-mapped database and extract location information from metadata

msa_name         = MSA_NAME
msa_file         = f'{msa_name}.fasta'
msa_path_cluster = ppath.join(SSH_DATA, msa_name, msa_name + '.fasta')
script_file      = 'merge-metadata-lmdb-parallel.py'

num_dirs  = int(15600000 / 500000) + 1

job_file         = 'job-merge-db.sh'
job_str          = f"""#!/bin/bash -l

#SBATCH --time=7-00:00:00
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=256G
#SBATCH --job-name=merge
#SBATCH -e ./MPL/out/merge-error-%a
#SBATCH -o ./MPL/out/merge-out-%a
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH --array=0-{num_dirs}

# load and activate conda environment
module load anaconda
conda activate align-env

python {script_file} --meta_file {os.path.join(SSH_DATA, METADATA_INTERIM)} --msa_file {msa_path_cluster} --lmdbDir {DATABASE_DIR} --refFile {REF_FILE} --dbNumber $SLURM_ARRAY_TASK_ID --metaNew {METADATA_NEW}
""" 

f = open(os.path.join(SARS_DIR, job_file), mode='w')
f.write('%s\n' % job_str)
f.close()

meta_path      = os.path.join(SARS_DIR, METADATA_INTERIM)
msa_path_local = os.path.join(SARS_DIR, msa_file)
print(f'scp {msa_path_local} {SSH_HOME}{SSH_DATA} &&')
print(f'scp {meta_path} {SSH_HOME}{SSH_DATA}')
print('scp %s/%s %s '   % (SARS_DIR, job_file, SSH_HOME))
print('')

# commands to execute on the cluster

print('sbatch %s' % job_file)
print('')

scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing-pipeline/msa_2023-05-29.fasta efinn@cluster.csb.pitt.edu:/net/dali/home/barton/efinn/SC2/2023-06-05 &&
scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing-pipeline/outputmeta.csv efinn@cluster.csb.pitt.edu:/net/dali/home/barton/efinn/SC2/2023-06-05
scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing-pipeline/job-merge-db.sh efinn@cluster.csb.pitt.edu: 

sbatch job-merge-db.sh



#### find gaps, subset regions

In [13]:
# find list of known gaps

ref_df    = pd.read_csv(os.path.join(DATA_DIR, f'ref-index-{DATA_DATE}.csv'))
ref_index = list(ref_df['ref_index'])
ref_sites = [dp.get_label_new(str(i) + '-A')[:-2] for i in ref_index]
variants  = [epsilon, alpha, beta, gamma, lambda_new, delta, twentyE_EU1, D614G, omicron, ba2, ba5, ba4, ba2121, kappa, mu, b12, b11318, b1429, B1_1_318]
gap_sites = []
for var in variants:
    for site in var:
        if site[-1]=='-':
            gap_sites.append(ref_index[ref_sites.index(site[:-2])])
for i in list('abcdefghijklmnopqrstuvwxyz'):
    gap_sites.append(f'22203{i}')
out_file = os.path.join(DATA_DIR, 'gap-list.npy')
np.save(out_file, gap_sites)

# transfer list of known gaps to the cluster

print(f'scp {out_file} {SSH_HOME}')

scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing/gap-list.npy efinn@cluster.csb.pitt.edu:


In [15]:
# finding the number of sequences in each region
out_file      = f'regional-{DATA_DATE}'
script_file   = 'check-regional.py'
job_dir       = SSH_HOME + SSH_DATA
job_file      = 'job-check-regional.sh'
job_str       = f"""#!/bin/bash -l

#SBATCH --time=0-3
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=100G
#SBATCH --job-name=regional-check
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH -e ./region-error
#SBATCH -o ./region-out
#SBATCH -p batch

python {script_file} --alignment {os.path.join(SSH_DATA, METADATA_INTERIM)} -o {os.path.join(SSH_DATA, out_file)}
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), mode='w')
f.write('%s\n' % job_str)
f.close()

# transfer data processing scripts and job file to the cluster

script_path = os.path.join(SCRIPT_DIR, script_file)
job_path = os.path.join(SCRIPT_DIR, job_file)
print(f'scp {script_path} {SSH_HOME} {SSH_DATA}')
print(f'scp {job_path} {SSH_HOME} {SSH_DATA}')
print('')

# run the job on the cluster

print('sbatch %s' % job_file)
print('')

# transfer output data back to local directory

out_path = os.path.join(job_dir, out_file + '.csv')
print('scp %s %s' % (out_path, SARS_DIR))

scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing-pipeline/SC2_BP_Data/processing-scripts/check-regional.py efinn001@cluster.hpcc.ucr.edu:/rhome/efinn001/shared/bigdata/deconvolution-2023-06-01/
scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing-pipeline/SC2_BP_Data/processing-scripts/job-check-regional.sh efinn001@cluster.hpcc.ucr.edu:/rhome/efinn001/shared/bigdata/deconvolution-2023-06-01/

sbatch job-check-regional.sh

scp efinn001@cluster.hpcc.ucr.edu:/rhome/efinn001/bigdata/SARS-CoV-2-Data/2023-06-05/regional-2023-06-05.csv /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing-pipeline/SC2_BP_Data


In [4]:
# Regions that are selected manually because either
# 1. They are broken up in an abnormal manner
# 2. For some of the pandemic the case numbers in the region were too low for the model to be reliable 
#(or there was essentially no community transmission)

north_cali = ['santa clara county, alameda county', 'san francisco county', 'sacramento county', 
              'contra costa county', 'humboldt county', 'sonoma county', 'stanislaus county', 
              'san joaquin county', 'solano county', 'merced county', 'santa cruz',
              'madera county', 'southern san joaquin valley', 'placer county',  
              'davis', 'nevada county', 'santa cruz county', 'san mateo county', 'butte county', 
              'mendocino county', 'shasta county', 'tuolumne county', 'napa county',
              'marin county', 'el dorado county', 'lake county', 'san luis obispo', 
              'sutter county', 'contra costa', 'tehama county']

south_cali = ['los angeles', 'los angeles county', 'orange', 'orange county', 'san diego', 
              'san diego county', 'ventura', 'ventura county', 'san luis obispo', 
              'san luis obispo county', 'imperial', 'imperial county', 'san bernardino county', 
              'riverside county', 'kern county', 'imperial county']

regs = [['europe', 'united kingdom', 'northern ireland', None, None, None],
        ['europe', 'united kingdom', ['england', 'wales', 'scotland'], None, None, None],
        ['north america', 'usa', 'california', south_cali, None, None],
        ['north america', 'usa', 'california', north_cali, None, None]]

state_elim   = ['california']
country_elim = []
for entry in regs:
    if entry[1]!='usa':
        country_elim.append(entry[1])

# Large countries or states, that must be broken up into smaller regions

use_subregions  = [          'usa',        'mexico',        'canada', 'turkey', 'india',        'brazil', 'russia',     'argentina',         'chile']
cont_subregions = ['north america', 'north america', 'north america',   'asia',  'asia', 'south america', 'europe', 'south america', 'south america']

In [5]:
df = pd.read_csv(os.path.join(SARS_DIR, f'regional-{DATA_DATE}.csv'))
continent = [str(i) for i in list(df['continent'])]
country   = [str(i) for i in list(df['country'])]
combined  = [continent[i] + '-' + country[i] for i in range(len(continent))]
df['combined'] = combined
data           = df['combined'].value_counts()

regions = np.array(list(data.axes[0]))
counts  = np.array(list(data))
mask    = counts > MIN_REGIONAL_SEQS
regions = regions[mask]
counts  = counts[mask]

regions    = [i.split('-') for i in regions]
continents = [i[0] for i in regions]
countries  = [i[1] for i in regions]

for i in range(len(regions)):
    if countries[i] not in use_subregions and countries[i] not in country_elim:
        regs.append([continents[i], countries[i], None, None, None, None])

for i in range(len(use_subregions)):
    continent = cont_subregions[i]
    country   = use_subregions[i]
    df_temp   = df[(df['continent']==continent) & (df['country']==country)]
    data      = df_temp['region'].value_counts()
    states    = np.array(list(data.axes[0]))
    counts    = np.array(list(data))
    
    mask      = counts > 1000
    states    = states[mask]
    counts    = counts[mask]
    
    for j in range(len(states)):
        if states[j] not in state_elim:
            regs.append([continent, country, states[j], None, None, None]) 

regs.sort(key = lambda x: x[:2])      
selected = regs

In [6]:
len(selected)

# not shown: manual sub-setting of 297 regions for sample set

297

In [12]:
regions_dir = os.path.join(SARS_DIR, f'regions-{DATA_DATE}')
if not os.path.exists(regions_dir):
    os.mkdir(regions_dir)

In [16]:
for group in selected:
    new = np.array(group)[np.array(group)!=None]
    contains_list = False
    for i in new:
        if not isinstance(i, str):
            contains_list = True
    if contains_list:
        label = ''
        for i in new:
            if isinstance(i, str):
                label += i
            else:
                if len(i) > 4:
                    #print(i)
                    if new[2]=='california':
                        if 'riverside county' in new[3]:
                            label += 'south'
                        else:
                            label += 'north'
                else:
                    label += '_'.join(i)
            if i!=new[-1]:
                label += '-'
    else:
        label = '-'.join(new)
    f = open(os.path.join(regions_dir, label + '.npy'), mode='wb')
    np.save(f, [group])
    f.close()

In [21]:
# Transfer scripts, run processing, and extract files

# Processing clips the time series in different regions according to set rules 
    # regarding the distribution of the number of sampled genomes over time

regions_dir_sub = os.path.join(DATA_DIR, f'regions-{DATA_DATE}-sub')
out_folder    = ppath.join(SSH_DATA, 'regions-times')
script_file   = 'find-region-times-parallel.py'
region_data_sub   = f'regions-{DATA_DATE}-sub'

job_file      = 'job-find-regions-parallel.sh'
job_str       = f"""#!/bin/bash -l 
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=50G
#SBATCH --time=1-00:00:00
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH -e ./logs/region-error-%a
#SBATCH -o ./logs/region-out-%a
#SBATCH --array=0-29

regions=({region_data_sub}/*)
region=${{regions[$SLURM_ARRAY_TASK_ID]}}
python {script_file} --input_file {ppath.join(SSH_DATA, METADATA_INT)} -o {out_folder} --regions \"$region\" 
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

job_path = os.path.join(SCRIPT_DIR, job_file)
script_path = os.path.join(SCRIPT_DIR, script_file)
print(f'scp {script_path} {SSH_HOME}{SSH_DATA} &&')
print(f'scp {job_path} {SSH_HOME}{SSH_DATA} &&')
print('scp -r %s %s/net/dali/home/barton/efinn/deconvolution-new' % (regions_dir_sub, SSH_HOME))
print('')

print('sbatch %s' % job_file)

scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/processing-scripts/find-region-times-parallel.py efinn@cluster.csb.pitt.edu:/net/dali/home/barton/efinn/deconvolution-new &&
scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/processing-scripts/job-find-regions-parallel.sh efinn@cluster.csb.pitt.edu:/net/dali/home/barton/efinn/deconvolution-new &&
scp -r /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing/regions-2024-01-26-sub efinn@cluster.csb.pitt.edu:/net/dali/home/barton/efinn/deconvolution-new

sbatch job-find-regions-parallel.sh


<a id="5"></a> 
## Processing
#### Extracting sequences linked to metadata, removing extreme genome gaps, imputation

In [28]:
# Processing with multiple lightning mapped databases storing the sequences

out_folder    = ppath.join(SSH_DATA, 'msa-reg')
script_file   = 'extract-seqs.py'
region_dir    = ppath.join(SSH_DATA, f'regions-times')
lmdb_dir      = SSH_LMDB

# run on cluster

print(f'ls {region_dir} | wc -l &&')
print('')
num_regions = 32

job_file      = 'job-extract.sh'
job_str       = f"""#!/bin/bash -l 
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=100G
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH -e ./logs/genome-error-%a
#SBATCH -o ./logs/genome-out-%a
#SBATCH --array=0-{num_regions - 1}

module purge
module load anaconda
eval "$(conda shell.bash hook)"
conda activate align-env

regions=({region_dir}/*)
region=${{regions[$SLURM_ARRAY_TASK_ID]}}
python {script_file} --input_file {METADATA_NEW} -o {out_folder} --regions \"$region\"  --lmdb {lmdb_dir}
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

# transfer job file to cluster

script_path = os.path.join(SCRIPT_DIR, script_file)
job_path = os.path.join(SCRIPT_DIR, job_file)
print(f'scp {script_path} {SSH_HOME}{SSH_DATA} &&')
print(f'scp {job_path} {SSH_HOME}{SSH_DATA}')
print('')

print('sbatch %s' % job_file)

ls /net/dali/home/barton/efinn/deconvolution-new/regions-times | wc -l &&

scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/processing-scripts/extract-seqs.py efinn@cluster.csb.pitt.edu:/net/dali/home/barton/efinn/deconvolution-new &&
scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/processing-scripts/job-extract.sh efinn@cluster.csb.pitt.edu:/net/dali/home/barton/efinn/deconvolution-new

sbatch job-extract.sh


In [4]:
# Additional processing and removing excess gaps

out_folder    = ppath.join(SSH_DATA, 'genome-data')
input_dir     = ppath.join(SSH_DATA, 'msa-reg')
script_file   = 'process-threaded.py'
threads       = 20

print(f'ls {input_dir} | wc -l &&')
print('')
num_regions = 30

job_file      = 'job-process.sh'
job_str       = f"""#!/bin/bash -l 
#SBATCH --cpus-per-task={threads}
#SBATCH --mem=150G
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH -e ./logs/genome-error-%a
#SBATCH -o ./logs/genome-out-%a
#SBATCH --array=0-{num_regions - 1}

files=({input_dir}/*)
file=${{files[$SLURM_ARRAY_TASK_ID]}}
python {script_file} --input_file \"$file\" -o {out_folder} --refFile {REF_FILE + '.csv'} --threads {threads}
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

script_path = os.path.join(SCRIPT_DIR, job_file)
print(f'scp {script_path} {SSH_HOME}')
print('')

print('sbatch %s' % job_file)

#scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing/ref-index-2024-01-26.csv efinn@cluster.csb.pitt.edu:/net/dali/home/barton/efinn/deconvolution-new

ls /net/dali/home/barton/efinn/deconvolution-new/msa-reg | wc -l &&

scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/processing-scripts/job-process.sh efinn@cluster.csb.pitt.edu:

sbatch job-process.sh


In [7]:
# Finding counts and imputing gaps

input_dir   = ppath.join(SSH_DATA, 'genome-data')
mut_count_dir     = ppath.join(SSH_DATA, 'mutant-counts')

mut_script    = 'find-mut-counts.py'
script_file = mut_script


# run on cluster to find number of files

print(f"""readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0) &&""")    # in order to find the number of files in the input directory
print('echo ${#files[*]}')
print('')

num_files     = 31   # the number of data files 

job_file      = 'job-mut-counts.sh'
job_str       = f"""#!/bin/bash -l 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --partition=big_memory
#SBATCH --mem=100G
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH -e ./logs/mut-counts-error-%a
#SBATCH -o ./logs/mut-counts-out-%a
#SBATCH --array=1-{num_files-1}

readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0)

module load anaconda

file=${{files[$SLURM_ARRAY_TASK_ID]}}
python {mut_script} --input_file \"$file\" -o {mut_count_dir} --refFile {REF_FILE + '.csv'}
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

job_path = os.path.join(SCRIPT_DIR, job_file)
print(f'scp {job_path} {SSH_HOME}{SSH_DATA} &&')
print('')

script_path = os.path.join(SCRIPT_DIR, script_file)
print(f'scp {script_path} {SSH_HOME}{SSH_DATA}')
print('')

print('sbatch %s' % job_file)

readarray -d '' files < <(find /net/dali/home/barton/efinn/deconvolution-new/genome-data ! -name "*sites*" -print0) &&
echo ${#files[*]}

scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/processing-scripts/job-mut-counts.sh efinn@cluster.csb.pitt.edu:/net/dali/home/barton/efinn/deconvolution-new &&

scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/processing-scripts/find-mut-counts.py efinn@cluster.csb.pitt.edu:/net/dali/home/barton/efinn/deconvolution-new

sbatch job-mut-counts.sh


In [11]:
# imputing gaps

input_dir   = ppath.join(SSH_DATA, 'genome-data')
mut_count_dir     = ppath.join(SSH_DATA, 'mutant-counts')
impute_dir = ppath.join(SSH_DATA, 'impute-gaps')

impute_script    = 'impute-gaps.py'
script_file = impute_script


# run on cluster to find number of files

print(f"""readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0) &&""")
print('echo ${#files[*]}')
print('')

num_files     = 31   # the number of data files 

job_file      = 'job-impute.sh'
job_str       = f"""#!/bin/bash -l 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --partition=big_memory
#SBATCH --mem=100G
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH -e ./logs/impute-error-%a
#SBATCH -o ./logs/impute-out-%a
#SBATCH --array=1-{num_files-1}

readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0)

module load anaconda

file=${{files[$SLURM_ARRAY_TASK_ID]}}
python {impute_script} --input \"$file\" -o {impute_dir} --mut_dir {mut_count_dir} --refFile {REF_FILE + '.csv'}
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

job_path = os.path.join(SCRIPT_DIR, job_file)
print(f'scp {job_path} {SSH_HOME}{SSH_DATA} &&')
print('')

script_path = os.path.join(SCRIPT_DIR, script_file)
print(f'scp {script_path} {SSH_HOME}{SSH_DATA}')
print('')

print('sbatch %s' % job_file)

readarray -d '' files < <(find /net/dali/home/barton/efinn/deconvolution-new/genome-data ! -name "*sites*" -print0) &&
echo ${#files[*]}

scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/processing-scripts/job-impute.sh efinn@cluster.csb.pitt.edu:/net/dali/home/barton/efinn/deconvolution-new &&

scp /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/processing-scripts/impute-gaps.py efinn@cluster.csb.pitt.edu:/net/dali/home/barton/efinn/deconvolution-new

sbatch job-impute.sh


<a id="6"></a>
## Back Projection Processing

<a id="7"></a>
### Step 1: take formatted (subprocessed) sequence file, divide into unique and non-unique sequences for back projection

In [ ]:
# divide into unique sequences (only one instance) and non-unique (multiple counts)

input_dir = ppath.join(SSH_DATA, 'impute-gaps')
unique_dir     = ppath.join(SSH_DATA, 'unique')
unique_script    = 'unique.py'
script_file = unique_script


# run on cluster to find number of 

print(f"""readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0) &&""")
print('echo ${#files[*]}')
print('')

num_files     = 31   # the number of data files 

job_file      = 'job-unique.sh'

job_str       = f"""#!/bin/bash -l 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --partition=big_memory
#SBATCH --mem=100G
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH -e ./logs/unique-error-%a
#SBATCH -o ./logs/unique-out-%a
#SBATCH --array=1-{num_files-1}

readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0)

module load anaconda

file=${{files[$SLURM_ARRAY_TASK_ID]}}
python {unique_script} --input \"$file\" -o {unique_dir}
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

job_path = os.path.join(SCRIPT_DIR, job_file)
print(f'scp {job_path} {SSH_HOME}{SSH_DATA} &&')
print('')

script_path = os.path.join(SCRIPT_DIR, script_file)
print(f'scp {script_path} {SSH_HOME}{SSH_DATA}')
print('')

print('sbatch %s' % job_file)

<a id="8"></a>
### step 2: transfer bp files to cluster, run deconvolution (back-projection) script

### Step 2a: setting up submission directory to run deconvolution (back-projection) 

In [29]:
# country_list.txt is a file with manually selected regions

os.chdir(SCRIPT_DIR)
with open('country_list.txt', 'r') as f:
    with open('country_list_command.txt', 'a') as w_f:
        for line in f:
            country = line.strip()
            command = f'ls {SSH_DATA}/genome-unique/{country}/output-{country} | wc -l'
            w_f.write(command + '\n')

In [64]:
df = pd.read_csv('countries_sh_maker.csv')

for index, row in df.iterrows():
    country = row['country']
    num_files = row['num_files'] #1638


    script_file = 'back_projection_ems_parallel.py'
    job_file = f'job-bp-ems-{country}.sh'
    job_str = f"""#!/bin/bash -l

#SBATCH --job-name=back-projection-{country}
#SBATCH --output=./bp_logs/bp_{country}_%A_%a.out
#SBATCH --error=./bp_logs/bp_{country}_%A_%a.err
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=2G
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH --array=0-{num_files}

echo Running on `hostname`
echo workdir $SLURM_SUBMIT_DIR

# Load necessary modules and activate environment
module load anaconda
conda activate align-env

# Define input and output directories
input_dir='{SSH_DATA}/genome-unique/{country}/output-{country}'
output_dir='{SSH_DATA}/genome-unique/{country}/bp-output-{country}'

# Create output directory if it does not exist
if [[ ! -d $output_dir ]]; then
mkdir -p $output_dir
fi

# Define the list of input files
input_files=("$input_dir"/*.csv)

# Get the input file for this job based on SLURM array task ID
input_file="${{input_files[$SLURM_ARRAY_TASK_ID]}}"

# Define scratch directory
SCRDIR=/scr/${{SLURM_JOB_ID}}

# Create scratch directory
if [[ ! -e $SCRDIR ]]; then
    mkdir $SCRDIR
fi

chmod +rX $SCRDIR

echo scratch drive ${{SCRDIR}}

cd $SCRDIR

# Copy input files to scratch directory
cp "$input_file" $SCRDIR
cp $SLURM_SUBMIT_DIR/back_projection_ems_parallel.py ${{SCRDIR}}
cp $SLURM_SUBMIT_DIR/incubation_period_sars2.csv ${{SCRDIR}}
cp $SLURM_SUBMIT_DIR/weights_binom.csv ${{SCRDIR}}

# Run the script from the scratch directory
cd $SCRDIR

# Run the script with the correct read directory
python back_projection_ems_parallel.py -input_file "$(basename "$input_file")" -write_dir "$output_dir" -incubation_file 'incubation_period_sars2.csv' -smoothing_file 'weights_binom.csv'

# Print job completion information
echo "SLURM job ID: $SLURM_JOB_ID"
echo "Array task ID: $SLURM_ARRAY_TASK_ID"
echo "Job completed at $(date)"
"""
    
    with open(os.path.join(SCRIPT_DIR, 'bp-submit-dir', job_file), 'w') as f:
        f.write(job_str)
    
    print('sbatch %s &&' % job_file)
        
print('scp -r %s/%s %s%s &&' % (SCRIPT_DIR, 'bp-submit-dir', SSH_HOME, SSH_DATA))
print('scp %s/%s %s%s' % (SCRIPT_DIR, script_file, SSH_HOME, SSH_DATA))


sbatch job-bp-ems-africa-cameroon.sh &&
sbatch job-bp-ems-africa-kenya.sh &&
sbatch job-bp-ems-africa-reunion.sh &&
sbatch job-bp-ems-africa-south-africa-2020-01-02-2022-11-25.sh &&
sbatch job-bp-ems-africa-south-africa-2022-11-25-2024-01-25.sh &&
sbatch job-bp-ems-asia-hong-kong.sh &&
sbatch job-bp-ems-asia-india-gujarat.sh &&
sbatch job-bp-ems-asia-india-karnataka.sh &&
sbatch job-bp-ems-asia-indonesia-2020-01-02-2023-03-31.sh &&
sbatch job-bp-ems-asia-indonesia-2023-03-31-2024-01-30.sh &&
sbatch job-bp-ems-asia-vietnam.sh &&
sbatch job-bp-ems-europe-bosnia-and-herzegovina.sh &&
sbatch job-bp-ems-europe-croatia.sh &&
sbatch job-bp-ems-europe-cyprus.sh &&
sbatch job-bp-ems-europe-estonia.sh &&
sbatch job-bp-ems-europe-latvia.sh &&
sbatch job-bp-ems-europe-portugal.sh &&
sbatch job-bp-ems-europe-romania.sh &&
sbatch job-bp-ems-europe-russia-moscow.sh &&
sbatch job-bp-ems-europe-slovakia.sh &&
sbatch job-bp-ems-north-america-canada-manitoba.sh &&
sbatch job-bp-ems-north-america-canada-q

<a id="9"></a>
### Step 3: reformatting to run downstream steps

### Step 3a: create format directory for slurm scripts

In [67]:
df = pd.read_csv('countries_sh_maker.csv')

for index, row in df.iterrows():
    country = row['country']
    
    script_file = 'format-deconvolution.py'
    job_file = f'job-format-{country}.sh'
    job_str = f"""#!/bin/bash

#SBATCH --job-name=bp-format-{country}
#SBATCH --output=./format_logs/format_{country}.out
#SBATCH --error=./format_logs/format_{country}.err
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=64G
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL

module load anaconda
conda activate align_env

output_dir='{SSH_DATA}/bp-formatted/'
input_dir='{SSH_DATA}'
sites_dir='{SSH_DATA}/impute-gaps/'
location='{country}'

python format-deconvolution.py -o "$output_dir" --input_dir "$input_dir" --sites_dir "$sites_dir" --location "$location" --prob_dist 'single-sequence-distribution.csv'
"""
    
    with open(os.path.join(SCRIPT_DIR, 'bp-format-sh-dir', job_file), 'w') as f:
        f.write(job_str)
    
    print('sbatch %s &&' % job_file)
        
print('scp -r %s/%s %s%s &&' % (SCRIPT_DIR, 'bp-format-sh-dir', SSH_HOME, SSH_DATA))
print('scp %s/%s %s%s' % (SCRIPT_DIR, script_file, SSH_HOME, SSH_DATA))

sbatch job-format-africa-cameroon.sh &&
sbatch job-format-africa-kenya.sh &&
sbatch job-format-africa-reunion.sh &&
sbatch job-format-africa-south-africa-2020-01-02-2022-11-25.sh &&
sbatch job-format-africa-south-africa-2022-11-25-2024-01-25.sh &&
sbatch job-format-asia-hong-kong.sh &&
sbatch job-format-asia-india-gujarat.sh &&
sbatch job-format-asia-india-karnataka.sh &&
sbatch job-format-asia-indonesia-2020-01-02-2023-03-31.sh &&
sbatch job-format-asia-indonesia-2023-03-31-2024-01-30.sh &&
sbatch job-format-asia-vietnam.sh &&
sbatch job-format-europe-bosnia-and-herzegovina.sh &&
sbatch job-format-europe-croatia.sh &&
sbatch job-format-europe-cyprus.sh &&
sbatch job-format-europe-estonia.sh &&
sbatch job-format-europe-latvia.sh &&
sbatch job-format-europe-portugal.sh &&
sbatch job-format-europe-romania.sh &&
sbatch job-format-europe-russia-moscow.sh &&
sbatch job-format-europe-slovakia.sh &&
sbatch job-format-north-america-canada-manitoba.sh &&
sbatch job-format-north-america-canada-q

<a id="10"></a>
# trim regions, filter sites
#### after formatting: trimming, filtering for sites of interest (frequencies above 5%)

<a id="11"></a>
### trim back projected regions, individual jobs

In [68]:
# creating sh files from list of countries

df = pd.read_csv('countries_sh_maker.csv')

for index, row in df.iterrows():
    country = row['country']
    
    script_file = 'trimming-bp.py'
    job_file = f'job-trim-{country}.sh'
    job_str = f"""#!/bin/bash

#SBATCH --job-name=bp-trim-{country}
#SBATCH --output=./trim_logs/trim_{country}.out
#SBATCH --error=./trim_logs/trim_{country}.err
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=64G
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL

module load anaconda
conda activate align_env

input_dir='{SSH_DATA}/bp-formatted/'
output_dir='{SSH_DATA}/bp-trim-20/'
location='{country}'

python trimming-bp.py --input_dir "$input_dir" -o "$output_dir" --location "$location" -x 20 --thresh 0.04 -n 5
"""
    
    with open(os.path.join(SCRIPT_DIR, 'bp-trim-sh-dir', job_file), 'w') as f:
        f.write(job_str)
    
    print('sbatch %s &&' % job_file)
        
print('scp -r %s/%s %s%s &&' % (SCRIPT_DIR, 'bp-trim-sh-dir', SSH_HOME, SSH_DATA))
print('scp %s/%s %s%s' % (SCRIPT_DIR, script_file, SSH_HOME, SSH_DATA))

sbatch job-trim-africa-cameroon.sh &&
sbatch job-trim-africa-kenya.sh &&
sbatch job-trim-africa-reunion.sh &&
sbatch job-trim-africa-south-africa-2020-01-02-2022-11-25.sh &&
sbatch job-trim-africa-south-africa-2022-11-25-2024-01-25.sh &&
sbatch job-trim-asia-hong-kong.sh &&
sbatch job-trim-asia-india-gujarat.sh &&
sbatch job-trim-asia-india-karnataka.sh &&
sbatch job-trim-asia-indonesia-2020-01-02-2023-03-31.sh &&
sbatch job-trim-asia-indonesia-2023-03-31-2024-01-30.sh &&
sbatch job-trim-asia-vietnam.sh &&
sbatch job-trim-europe-bosnia-and-herzegovina.sh &&
sbatch job-trim-europe-croatia.sh &&
sbatch job-trim-europe-cyprus.sh &&
sbatch job-trim-europe-estonia.sh &&
sbatch job-trim-europe-latvia.sh &&
sbatch job-trim-europe-portugal.sh &&
sbatch job-trim-europe-romania.sh &&
sbatch job-trim-europe-russia-moscow.sh &&
sbatch job-trim-europe-slovakia.sh &&
sbatch job-trim-north-america-canada-manitoba.sh &&
sbatch job-trim-north-america-canada-quebec-2020-01-02-2022-08-08.sh &&
sbatch job

<a id="12"></a>
### trimming naive
#### for comparison

In [ ]:
# trimming step, naive: finding time series with good sampling

input_dir     = ppath.join(SSH_DATA, 'genome-data')
output_dir    = ppath.join(SSH_DATA)
script_file   = 'trim-intervals-sd.py'

print(f"""readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0)""")
print('echo ${#files[*]}')
print('')

job_file      = 'job-trim-intervals.sh'
job_str       = f"""#!/bin/bash -l 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --partition=big_memory
#SBATCH -e ./sampling-error-%a
#SBATCH -o ./sampling-out-%a
#SBATCH --array=1-5

readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0)
file=${{files[$SLURM_ARRAY_TASK_ID]}}

python {script_file} --input \"$file\" -o {output_dir} --trimDir genome-trimmed
""" 

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

script_path = os.path.join(SCRIPT_DIR, job_file)
print(f'scp {script_path} {SSH_HOME}')
print('')

print('sbatch %s' % job_file)

<a id="13"></a>
### finding max frequencies and filtering
#### for comparison with back projection

<a id="14"></a>
### step 1: find naive max freq

In [ ]:
# find max, naive

input_dir = ppath.join(SSH_DATA, 'genome-trimmed')
max_dir = ppath.join(SSH_DATA, 'genome-max-freqs')

max_freq_script = 'find-max-frequencies.py'
script_file = max_freq_script

# run on cluster to find number of files

print(f"""readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0) &&""")
print('echo ${#files[*]}')
print('')

num_files     = 180   # the number of data files 

job_file      = 'job-max-freq.sh'

job_str       = f"""#!/bin/bash -l
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=100G
#SBATCH -e ./logs/freqs-error-%a
#SBATCH -o ./logs/freqs-out-%a
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH --array=1-{num_files-1}

readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0)

file=${files[$SLURM_ARRAY_TASK_ID]}
python {max_freq_script} --input_file "$file" -o {max_dir} --refFile {REF_FILE + '.csv'}
"""

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

job_path = os.path.join(SCRIPT_DIR, job_file)
print(f'scp {job_path} {SSH_HOME}{SSH_DATA} &&')
print('')

script_path = os.path.join(SCRIPT_DIR, script_file)
print(f'scp {script_path} {SSH_HOME}{SSH_DATA}')
print('')

print('sbatch %s' % job_file)

<a id="15"></a>
### step 2 find masks and filter naive

In [ ]:
# individual masks: mask off sites for time-regions that rise above 5% frequency in naive method

input_dir = ppath.join(SSH_DATA, 'genome-trimmed')
filtered_dir = ppath.join(SSH_DATA, 'genome-filtered-05')
max_dir = ppath.join(SSH_DATA, 'genome-max-freqs')
max_mask_dir = ppath.join(SSH_DATA, 'genome-max-mask-05')

filter_mask_script    = 'filter-sites-mask.py'
script_file = filter_mask_script

# run on cluster to find number of files

print(f"""readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0) &&""")
print('echo ${#files[*]}')
print('')

num_files     = 180   # the number of data files 

job_file      = 'job-filter-sites-mask-05.sh'

job_str       = f"""#!/bin/bash -l 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=56G
#SBATCH -e ./logs/mask-error-%a
#SBATCH -o ./logs/mask-out-%a
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH --array=1-{num_files-1}

module load anaconda

readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0)
file=${files[$SLURM_ARRAY_TASK_ID]}
python {filter_mask_script} --input "$file" -o {filtered_dir} --max_dir {max_dir} --mask_dir {max_mask_dir} --refFile {REF_FILE + '.csv'} --min_freq 0.05
"""

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

job_path = os.path.join(SCRIPT_DIR, job_file)
print(f'scp {job_path} {SSH_HOME}{SSH_DATA} &&')
print('')

script_path = os.path.join(SCRIPT_DIR, script_file)
print(f'scp {script_path} {SSH_HOME}{SSH_DATA}')
print('')

print('sbatch %s' % job_file)

<a id="16"></a>
### step 3: combine masks

In [ ]:
# combined masks (incomplete; run locally) 
# TO DO: make py script
# creates a common mask for time-regions in each region

def get_region_name(file_name):
    # extract region filename
    return file_name.split('---')[0]

def print_true_indices(mask, label):
    indices = np.argwhere(mask)
    print(f"\n{label} true indices:")
    for index in indices:
        print(tuple(index))

def load_combine_masks(file_list, region):
    combined_mask = None
    for file in file_list:
        mask = np.load(file)
        # print (optional for debug) true indices for individual mask
        print_true_indices(mask, f"{region} - {file}")
        if combined_mask is None:
            combined_mask = mask
        else:
            combined_mask = np.logical_or(combined_mask, mask)
    return combined_mask

# main function to run in notebook
def combine_main(directory):
    mask_files = []
    
    # directory walk for mask files
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('-mask.npy'):
                mask_files.append(os.path.join(root, file))
    
    # group region files
    region_files = defaultdict(list)
    for file in mask_files:
        region = get_region_name(os.path.basename(file))
        region_files[region].append(file)
    
    # combine mask
    combined_masks = {}
    for region, files in region_files.items():
        combined_mask = load_combine_masks(files, region)
        combined_masks[region] = combined_mask
        # print (optional, remove if not needed)
        print_true_indices(combined_mask, f"{region} - combined mask")
    
    # save combined mask
    for region, mask in combined_masks.items():
        output_file = os.path.join(directory, f"{region}-combined-mask.npy")
        np.save(output_file, mask)
        print(f"Combined mask saved to: {output_file}")

In [ ]:
# run for combined masks to get comparison frequency files

input_dir = ppath.join(SSH_DATA, 'genome-trimmed')
filter_comparison_dir = ppath.join(SSH_DATA, 'genome-filtered-compare-05')
combine_mask_dir = ppath.join(SSH_DATA, 'genome-max-combine-mask-05')
    # combine mask directory made locally and uploaded from previous cell

filter_combine_script    = 'filter-sites-combine-mask.py'
script_file = filter_combine_mask_script

# run on cluster to find number of 

print(f"""readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0) &&""")
print('echo ${#files[*]}')
print('')

num_files     = 180   # the number of data files 

job_file      = 'job-filter-sites-compare-mask-05.sh'
job_str       = f"""#!/bin/bash -l 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=32G
#SBATCH -e ./logs/mask-combine-error-%a
#SBATCH -o ./logs/mask-combine-out-%a
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH --array=1-{num_files-1}

module load anaconda

readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0)
file=${{files[$SLURM_ARRAY_TASK_ID]}}

python {filter_combine_script} --input "$file" -o {filter_comparison_dir} --mask_dir {combine_mask_dir} --refFile {REF_FILE + '.csv'}
"""

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

job_path = os.path.join(SCRIPT_DIR, job_file)
print(f'scp {job_path} {SSH_HOME}{SSH_DATA} &&')
print('')

script_path = os.path.join(SCRIPT_DIR, script_file)
print(f'scp {script_path} {SSH_HOME}{SSH_DATA}')
print('')

print('sbatch %s' % job_file)

<a id="17"></a>
### step 4: subset back projection site files with previously made masks 

In [ ]:
# individual masks

input_dir   = ppath.join(SSH_DATA, 'bp-trim-20')
bp_filter_dir = ppath.join(SSH_DATA, 'bp-filter-20-05')
max_mask_dir = ppath.join(SSH_DATA, 'genome-max-mask-05')

bp_filter_script    = 'filter-sites-bp-mask.py'
script_file = bp_filter_script

# run on cluster to find number of files

print(f"""readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0) &&""")
print('echo ${#files[*]}')
print('')

num_files      = 180

job_file       = 'job-filter-bp-sites-mask-05.sh'
job_str        = f"""#!/bin/bash 
#SBATCH --job-name=filt-bp-mask
#SBATCH --nodes=1
#SBATCH --mem=50G
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH -e ./logs/bp-filt-error-%a
#SBATCH -o ./logs/bp-filt-out-%a
#SBATCH --array=1-{num_files-1}


readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0)
file=${{files[$SLURM_ARRAY_TASK_ID]}}

python {bp_filter_script} -o {bp_filter_dir} --input "$file" --mask_dir {max_mask_dir} --refFile {REF_FILE + '.csv'}
"""

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

job_path = os.path.join(SCRIPT_DIR, job_file)
print(f'scp {job_path} {SSH_HOME}{SSH_DATA} &&')
print('')

script_path = os.path.join(SCRIPT_DIR, script_file)
print(f'scp {script_path} {SSH_HOME}{SSH_DATA}')
print('')

print('sbatch %s' % job_file)

In [ ]:
# combined masks

input_dir   = ppath.join(SSH_DATA, 'bp-trim-20')
bp_filter_compare_dir = ppath.join(SSH_DATA, 'bp-filtered-compare-05')
combine_mask_dir = ppath.join(SSH_DATA, 'genome-max-combine-mask-05')

bp_filter_combine_script    = 'filter-sites-bp-combine-mask.py'
script_file = bp_filter_script

# run on cluster to find number of files

print(f"""readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0) &&""")
print('echo ${#files[*]}')
print('')

num_files      = 149

job_file       = 'job-filter-bp-sites-compare-mask-05.sh'
job_str        = f"""#!/bin/bash -l
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=32G
#SBATCH -e ./logs/bp-mask-combine-error-%a
#SBATCH -o ./logs/bp-mask-combine-out-%a
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH --array=1-{num_files-1}


readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0)
file=${{files[$SLURM_ARRAY_TASK_ID]}}

python {bp_filter_combine_script} -o {bp_filter_compare_dir} --input "$file" --mask_dir {combine_mask_dir} --refFile {REF_FILE + '.csv'}
"""

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

job_path = os.path.join(SCRIPT_DIR, job_file)
print(f'scp {job_path} {SSH_HOME}{SSH_DATA} &&')
print('')

script_path = os.path.join(SCRIPT_DIR, script_file)
print(f'scp {script_path} {SSH_HOME}{SSH_DATA}')
print('')

print('sbatch %s' % job_file)

<a id="18"></a>
### Covariance and Inference

This section is a bit ad hoc.

<a id="19"></a>
### covariance, cluster

In [ ]:
input_dir = ppath.join(SSH_DATA, 'bp-filtered-compare-05')
temp_dir = ppath.join(SSH_DATA, 'bp-covar')
scratch_dir =  ppath.join(SSH_DATA, 'scratch-bp-covar')

covar_bp_script    = 'epi-covar-bp-cluster.py'
script_file = covar_bp_script

# run on cluster to find number of files

print(f"""readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0) &&""")
print('echo ${#files[*]}')
print('')

num_files      = 149

job_file       = 'job-epi-covar-bp.sh'

job_str        = f"""#!/bin/bash -l

#!/bin/bash -l 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=100G
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH -e ./logs/mpl/bp-freq-error-%a
#SBATCH -o ./logs/mpl/bp-freq-out-%a
#SBATCH --array=1-{num_files-1}%1

cd ./inference-c++
readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0)
g++ src/main-N.cpp src/inf.cpp src/io.cpp -O3 -march=native -lgslcblas -lgsl -o bin/mpl -std=c++11

file=${{files[$SLURM_ARRAY_TASK_ID]}}
tempout={temp_dir}
python ../{covar_bp_script} --data "$file" -o $tempout -q 5 --pop_size 10000 -k 0.1 -R 2 --scratch {scratch_dir} --timed 1
"""

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

job_path = os.path.join(SCRIPT_DIR, job_file)
print(f'scp {job_path} {SSH_HOME}{SSH_DATA} &&')
print('')

script_path = os.path.join(SCRIPT_DIR, script_file)
print(f'scp {script_path} {SSH_HOME}{SSH_DATA}')
print('')

print('sbatch %s' % job_file)

#### covariance, naive

In [ ]:
input_dir = ppath.join(SSH_DATA, 'genome-filtered-compare-05')
temp_dir = ppath.join(SSH_DATA, 'genome-covar')
scratch_dir =  ppath.join(SSH_DATA, 'scratch-genome-covar')

covar_script    = 'epi-covar-cluster.py'
script_file = covar_script

# run on cluster to find number of files

print(f"""readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0) &&""")
print('echo ${#files[*]}')
print('')

num_files      = 180

job_file       = 'job-epi-covar.sh'

job_str        = f"""#!/bin/bash -l 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=100G
#SBATCH --mail-user={USER_EMAIL}
#SBATCH --mail-type=ALL
#SBATCH -e ./logs/mpl/freq-error-%a
#SBATCH -o ./logs/mpl/freq-out-%a
#SBATCH --array=1-{num_files-1}%1

cd ./inference-c++
readarray -d '' files < <(find {input_dir} ! -name "*sites*" -print0)
g++ src/main-N.cpp src/inf.cpp src/io.cpp -O3 -march=native -lgslcblas -lgsl -o bin/mpl -std=c++11

file=${files[$SLURM_ARRAY_TASK_ID]}
tempout={temp_dir}
python ../{covar_script} --data "$file" -o $tempout -q 5 --pop_size 10000 -k 0.1 -R 2 --scratch {scratch_dir} --timed 1
"""

f = open(os.path.join(SCRIPT_DIR, job_file), 'w')
f.write('%s\n' % job_str)
f.close()

job_path = os.path.join(SCRIPT_DIR, job_file)
print(f'scp {job_path} {SSH_HOME}{SSH_DATA} &&')
print('')

script_path = os.path.join(SCRIPT_DIR, script_file)
print(f'scp {script_path} {SSH_HOME}{SSH_DATA}')
print('')

print('sbatch %s' % job_file)

<a id="20"></a>
### inference: local running on cluster files

In [ ]:
inf_script    = 'epi-inf-parallel.py'
archive_loc   = os.path.join(DATA_DIR, 'inference-c++')
covar_home    = os.path.join(DATA_DIR, 'bp-covar-south-africa')

os.chdir(DATA_DIR)

covar_country_target = os.path.join(covar_home, 'africa-south-africa')

for i in os.listdir(covar_country_target):
    if i == ".DS_Store":
            continue
    covar_dir      = os.path.join(covar_country_target, f'{i}')
    out_file       = os.path.join(home_dir, 'inference-south-africa', f'{i}-inf-out')
    %run {inf_script} --data {covar_dir} --timed 1 -o {out_file} -q 5 --g1 40 --saveCovar --refFile {REF_FILE}

<a id="21"></a>
### local running, cluster script not supported

In [ ]:
# FOR MACS

homebrew run command:  g++ src/main-N.cpp src/inf.cpp src/io.cpp -O3 -march=native -lgslcblas -lgsl -L/opt/homebrew/Cellar/gsl/2.7.1/lib -o bin/mpl -std=c++11

In [4]:
# infering the selection coefficients using a cutoff frequency of 5%
import os

freq_script = 'epi-covar-bp.py'
inf_script  = 'epi-inf-parallel.py'

filter_input_file = 'Europe-Portugal---2021-4-20-2022-11-6.csv'


out_inf_dir   = f'{DATA_DIR}/inf-bp-out'
#in_filt_dir   = f'{DATA_DIR}/filt-bp-20'


os.chdir(PROCESS_DIR)

archive_loc    = os.path.join(INF_DIR)
temp_dir       = os.path.join(PROCESS_DIR, 'temp')
out_file       = os.path.join(out_inf_dir, f'infer-Europe-Portugal---2021-4-20-2022-11-6.csv')

In [5]:
%run epi-covar-bp --data {filter_input_file} -o {temp_dir} -q 5 --pop_size 10000 -k 0.1 -R 2 --scratch {temp_dir}

scratch directory:    /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing-pipeline/temp
covariance directory: /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing-pipeline/temp/temp-sd-covar-dir
outfile /Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing-pipeline/temp
sequence file	 Europe-Portugal---2021-4-20-2022-11-6.csv
sequence lengths are [76]
number of sequences with each length [129586]
[  685   686   687   688   689   690   691   692   693  6512  6513  6514
 11282 11283 11284 11285 11286 11287 11288 11289 11290 11291 11292 11293
 11294 11295 21632 21633 21634 21635 21636 21637 21638 21639 21640 21764
 21765 21766 21767 21768 21769 21986 21987 21988 21989 21990 21991 21992
 21993 21994 22028 22029 22030 22031 22032 22033 22193 22194 22195 27888
 28247 28248 28249 28250 28251 28252 28270 28361 28362 28363 28364 28365
 28366 28367 28368 28369]
number of sites:	76
        

In [6]:
%run {inf_script} --data {temp_dir} --timed 1 -o {out_inf_dir} -q 5 --g1 40 --saveCovar --refFile {'ref-index-2023-06-05.csv'}

starting
/Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing-pipeline/temp
.DS_Store
/Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing-pipeline/temp
Europe-Portugal---2021-4-20-2022-11-6.npz
/Users/liz/Documents/back_projection_home/backprojection-SARS-CoV-2-main/data-processing-pipeline/temp/Europe-Portugal---2021-4-20-2022-11-6.npz
	loading location Europe-Portugal---2021-4-20-2022-11-6
[  685   686   687   688   689   690   691   692   693  6512  6513  6514
 11282 11283 11284 11285 11286 11287 11288 11289 11290 11291 11292 11293
 11294 11295 21632 21633 21634 21635 21636 21637 21638 21639 21640 21764
 21765 21766 21767 21768 21769 21986 21987 21988 21989 21990 21991 21992
 21993 21994 22028 22029 22030 22031 22032 22033 22193 22194 22195 27888
 28247 28248 28249 28250 28251 28252 28270 28361 28362 28363 28364 28365
 28366 28367 28368 28369]
Europe-Portugal
number of sites:		76
[[  0.         952.38095238   0